<a href="https://colab.research.google.com/github/desultir/assign1/blob/master/assign2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
  
#!cd "/content/drive/My Drive/Colab Notebooks/PretrainedModels" && tar xvzf *.tar.gz
#!cd "/content/drive/My Drive/Colab Notebooks/Input" && tar xvzf test.tar.gz

Mounted at /content/drive


In [2]:
#install tensorboard tunnel
#https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
#!cd "/content/drive/My Drive/Colab Notebooks/Input/val2014" && cp * ../../LabelingVal/

--2019-05-23 03:17:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.7.169.168, 34.206.130.40, 52.4.95.48, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.7.169.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  15.88M  38.4MB/s    in 0.4s    

2019-05-23 03:17:11 (38.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
MODEL_DIR = "/content/drive/My Drive/Colab Notebooks/PretrainedModels/"
MODEL_NAME = "faster_rcnn_resnet50_coco_2018_01_28"

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [2]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://1a166d8b.ngrok.io


In [0]:
from matplotlib.image import imread
import PIL.Image
from PIL import ImageOps
import numpy as np
import io

def normalize_image(rgb):
  white_rgb = rgb - np.mean(rgb)
  white_rgb = white_rgb / np.max(np.abs(white_rgb))
  return white_rgb

def decompress_expand(jpg):
  #zero pad to 320x320
  encoded_jpg_io = io.BytesIO(jpg)
  image = PIL.Image.open(encoded_jpg_io)
  DESIRED_SIZE = 320
  widthpad = (DESIRED_SIZE - image.width)
  heightpad = (DESIRED_SIZE - image.height)
  padding = (widthpad//2, heightpad//2, widthpad - widthpad//2, heightpad-heightpad//2)
  padded = ImageOps.expand(image, padding)
  return padded


def load_and_preprocess(path):
  with tf.gfile.GFile(path, 'rb') as fp:
    jpg = fp.read()
  return decompress_expand(jpg)

def load_np(paths):
  return np.stack([imread(x) for x in paths])

In [0]:
import glob
import csv
import tensorflow as tf
import os
from tensorflow.core.framework import graph_pb2
from tensorflow.python.saved_model import loader
AUTOTUNE = tf.data.experimental.AUTOTUNE

NUM_TRAINING = 31924

#load train images
def get_training_images(split=False):
  #train_img_list = glob.glob(r"/content/drive/My Drive/Colab Notebooks/Input/train2014/*.jpg")
  train_img_list = ["/content/drive/My Drive/Colab Notebooks/Input/train2014/{}.jpg".format(i) for i in range(31924)]
  if not split:
    return train_img_list
  else:
    np.random.seed(1)
    np.random.shuffle(train_img_list)
    # 90/10 split
    split_pt = (NUM_TRAINING // 10) * 9
    return train_img_list[:split_pt], train_img_list[split_pt:]

#train_img_list_ds = tf.data.Dataset.from_tensor_slices(image_list)
#load train labels
train_labels_filename = r"/content/drive/My Drive/Colab Notebooks/Input/train.txt"
with open(train_labels_filename) as f:
  reader = csv.reader(f, delimiter='\t')
  train_labels = {k:list(map(int, v.split(','))) for k, v in reader}
  
def get_label(filename):
  if '/' in filename:
    filename = filename.rsplit('/', 1)[-1]
  return train_labels.get(filename)


In [9]:
#load the trainable model from the checkpoint
sess = tf.Session()

saver = tf.train.import_meta_graph(os.path.join(MODEL_DIR,MODEL_NAME, 'model.ckpt.meta'))

sess.run(tf.global_variables_initializer())
saver.restore(sess, os.path.join(MODEL_DIR,MODEL_NAME, 'model.ckpt'))



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/PretrainedModels/faster_rcnn_resnet50_coco_2018_01_28/model.ckpt


In [0]:
# write graph for tensorboard
summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

In [0]:

#PREDICTION
BATCH_SIZE = 10
batch_no = 0
batch_list = []
result_list = []
while image_list:
  try:
    while len(batch_list) < BATCH_SIZE:
      batch_list.append(next(image_list))
  except StopIteration:
    pass
  stacked_imgs = np.stack(batch_list)
  batch_result = sess.run(("detection_classes:0", "detection_scores:0"), feed_dict={"image_tensor:0": stacked_imgs})
  batch_list = []
  batch_no += 1
  print(batch_no)
  result_list.append(batch_result)
  
  
#result = sess.run(("SecondStagePostprocessor/convert_scores:0", "detection_classes:0", "num_detections:0"), feed_dict={"image_tensor:0": stacked_imgs})

In [0]:
import pickle
with open("/content/drive/My Drive/Colab Notebooks/full_results.pk", 'wb') as f:
  pickle.dump(result_list, f)

NameError: ignored

In [0]:
class relabeler(object):
  def __init__(self):
    self.labels = {}
     #mapping from cocolabel to tutorlabel + 1 (0 reserved for 'reject bounding box')
    self.labels[9] = 1 #boat 
    self.labels[10] = 5 #traffic light 
    self.labels[14] = 6 #parking meter
    self.labels[15] = 7 #bench
    self.labels[16] = 18 #bird
    self.labels[28] = 19 #umbrella
    self.labels[31] = 20 #handbag
    self.labels[36] = 2 #snowboard
    self.labels[39] = 4 #baseball bat
    self.labels[41] = 3 #skateboard
    self.labels[44] = 10 #bottle
    self.labels[49] = 9 #knife
    self.labels[50] = 8 #spoon
    self.labels[52] = 15 #banana
    self.labels[54] = 13 #sandwich
    self.labels[57] = 14 #carrot
    self.labels[79] = 17 #oven
    self.labels[80] = 16 #toaster
    self.labels[85] = 12 #clock
    self.labels[90] = 11 #toothbrush   
    self.fix_fn = self.get_fix_fn()
    
  def fix(self, old):
    return self.labels.get(old, 0)
  
  def get_fix_fn(self):
    return np.vectorize(self.fix)
    
  def get_best_label(self, preds):
    #convert to new label space and get best label
    labels = self.fix_fn(preds)
    #return labels
    #return first that isn't 0
    if len(np.nonzero(labels)[0]):
      return labels[np.nonzero(labels)[0][0]] - 1 #shift back into original label space
    else:
      return 19 #-1 handbag is the thing we most frequently fail at 

In [0]:
import pickle
#unpickle resulst file
with open("/content/drive/My Drive/Colab Notebooks/full_results.pk", 'rb') as f:
  results_list = pickle.load(f)
  
len(results_list)

3193

In [0]:
import numpy as np
results = [x[0] for x in results_list]
stacked_results = np.vstack(results)
stacked_results.shape

(31925, 100)

In [0]:
#convert preds back into tutor space
fixer = relabeler()
preds  = stacked_results
labels = list(map(fixer.get_best_label, preds))


In [11]:
from collections import Counter
correct = 0
labelled = zip(train_img_list, labels)
classes = Counter() # (pred, actual) : count

y_true = []
y_pred = []

for fn, pred in labelled:
  truth = train_labels[fn.split("/")[-1]]
  for truelabel in truth:
    classes[truelabel] += 1
  if pred in truth:
    correct += 1
    y_true.append(pred)
    y_pred.append(pred)
  else:
    for truelabel in truth: 
      y_true.append(truelabel)
      y_pred.append(pred)
    #print(fn, pred, truth)

    
print(correct, len(train_img_list))
print(classes)

NameError: ignored

In [31]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

#reference https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(15,15))
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=[x for x in range(-1,20)],
                      title='Confusion matrix')


NameError: ignored

In [12]:
## TRANSFER LEARNING
# add a single dense relu layer followed by softmax
label_pl = tf.placeholder(tf.float32, [None, 20], name='labels')
with tf.name_scope("train"):
  flat_scores = tf.layers.flatten(sess.graph.get_tensor_by_name("SecondStagePostprocessor/convert_scores:0"))
  #first_layer = tf.layers.dense(inputs=flat_scores, units=100*20, name="first_layer", activation=tf.nn.relu)
  #second_layer = tf.layers.dense(inputs=first_layer, units=25*20, name="second_layer", activation=tf.nn.relu)
  output = tf.layers.dense(inputs=flat_scores, units=20, name="output", activation=tf.nn.relu)


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=label_pl))

# Train op
#with tf.name_scope("train"):
# Get gradients of all trainable variables
# TODO actually pass vars in, not their names
var_list = [v for v in tf.trainable_variables()]
gradients = tf.gradients(loss, var_list, unconnected_gradients='none')
gradients = list(zip(gradients, var_list))
learning_rate = 0.01

# Create optimizer and apply gradient descent to the trainable variables
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.apply_gradients(grads_and_vars=gradients)

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [13]:
import time
for gradient, var in gradients:
    tf.summary.histogram(var.name + '/gradient', gradient)
    
for var in var_list:
    tf.summary.histogram(var.name, var)
    
tf.summary.scalar('cross_entropy', loss)

# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(label_pl, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Merge all summaries together
merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
writer = tf.summary.FileWriter(LOG_DIR + "/" + str(time.time()))

# Initialize an saver for store model checkpoints
saver = tf.train.Saver()




INFO:tensorflow:Summary name output/kernel:0/gradient is illegal; using output/kernel_0/gradient instead.
INFO:tensorflow:Summary name output/bias:0/gradient is illegal; using output/bias_0/gradient instead.
INFO:tensorflow:Summary name output/kernel:0 is illegal; using output/kernel_0 instead.
INFO:tensorflow:Summary name output/bias:0 is illegal; using output/bias_0 instead.


In [0]:
def MHE(labels, max_label=20):
  output = np.zeros(max_label, dtype=np.float32)
  for lab in labels:
    output[lab] = 1.0
    
  return output


In [0]:
from datetime import datetime
BATCH_SIZE = 32
num_epochs = 10

batch_no = 0

checkpoint_path = "/content/drive/My Drive/Colab Notebooks/Models/"
result_list = []
init = tf.global_variables_initializer()
sess.run(init)

train, val = get_training_images(True)
# Get the number of training/validation steps per epoch
train_batches_per_epoch = int(np.floor(len(train)/BATCH_SIZE))
val_batches_per_epoch = int(np.floor(len(val)/ BATCH_SIZE))
display_step = 50

def load_batch(from_iter):
  batch_list = []
  labels_list= []
  while len(batch_list) < BATCH_SIZE:
    nextfile = next(from_iter)
    batch_list.append(load_and_preprocess(nextfile))
    labels_list.append(MHE(get_label(nextfile)))
  return np.stack(batch_list), np.stack(labels_list)

for epoch in range(num_epochs):
  train_iter  = iter(train) 
  val_iter = iter(val)
  for step in range(train_batches_per_epoch):
    stacked_imgs, labels = load_batch(train_iter)
    batch_result = sess.run((train_op, loss), feed_dict={"image_tensor:0": stacked_imgs, "labels:0":labels})
    #result_list.append(batch_result)
    # Generate summary with the current batch of data and write to file
    print(".", end="")
    if step % display_step == 0:
        s = sess.run(merged_summary, feed_dict={"image_tensor:0": stacked_imgs, "labels:0":labels})

        writer.add_summary(s, epoch*train_batches_per_epoch + step)

  print("{} Start validation".format(datetime.now()))

  test_acc = 0.
  test_count = 0
  for _ in range(val_batches_per_epoch):
      stacked_imgs, labels = load_batch(val_iter)
      acc = sess.run(accuracy, feed_dict={"image_tensor:0": stacked_imgs, "labels:0":labels})
      test_acc += acc
      test_count += 1
  test_acc /= test_count
  print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                 test_acc))
  print("{} Saving checkpoint of model...".format(datetime.now()))

  # save checkpoint of the model
  checkpoint_name = os.path.join(checkpoint_path,
                                 'model_epoch'+str(epoch+1)+'.ckpt')
  save_path = saver.save(sess, checkpoint_name)

  print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                 checkpoint_name))
  
  
#result = sess.run(("SecondStagePostprocessor/convert_scores:0", "detection_classes:0", "num_detections:0"), feed_dict={"image_tensor:0": stacked_imgs})

..........

In [17]:
sess.graph.get_tensor_by_name("train/output:0")

KeyError: ignored

In [18]:
output

<tf.Tensor 'train/output/Relu:0' shape=(?, 20) dtype=float32>